In [ ]:
import matplotlib.pyplot as plt
import torch

from upper.geometry import compute_area
from upper.network import SofaNet

torch.set_default_dtype(torch.float64)
plt.rcParams['text.usetex'] = True

In [ ]:
# load model
model = SofaNet([128, 128, 128])
model.load_state_dict(torch.load("figs/upper_model.pt"))

# compute area
a1 = 0.283794109  # asin(7/25)
a2 = 0.532504098  # asin(33/65)
a3 = 0.781214087  # asin(119/169)
a4 = 1.03829223  # asin(56/65)
a5 = 1.28700222  # asin(24/25)
alpha = torch.tensor([a1, a2, a3, a4, a5])
u1, u2 = model.forward(alpha)
area = compute_area(alpha, torch.pi / 2, torch.pi / 2, u1, u2, n_areas=10000)
print(area.item())

In [ ]:
# the landscapes of `area` (Jaehoon)

# NOTE: landscape with respect to u1 and u2 instead of network weights